In [37]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import rasterio as rio

In [105]:
# Init Sentinel API

username = 'cryovision' 
password = 'baw.YFV8rfa8nva9mfg' 
api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

In [106]:
# Import Shapefile

cutoff = 2
pingo = gpd.read_file('pingo_distribution_siberia_ggrosse_bjones/pingo_distribution_siberia_ggrosse_bjones.shp')
pingo = pingo.to_crs('epsg:4326')
pingo = pingo.iloc[0:cutoff]
pingo.to_file('pingo.geojson', driver='GeoJSON')
footprint = geojson_to_wkt(read_geojson('pingo.geojson'))

In [78]:
# Get Images from api

images = api.query(footprint,
                        platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10),
                     limit=10)
dataframe = api.to_geodataframe(images)

In [ ]:
# # Download Image Data set using Id
# api.download("5ebfe45d-bec0-4655-80fd-6aef1b5f24d4")

In [35]:
R10 = 'S2A_MSIL2A_20220330T025551_N0400_R032_T52WEV_20220330T054426.SAFE/GRANULE/L2A_T52WEV_A035350_20220330T025547/IMG_DATA/R10m'
b4 = rio.open(R10+'/T52WEV_20220330T025551_B04_10m.jp2')
b3 = rio.open(R10+'/T52WEV_20220330T025551_B03_10m.jp2')
b2 = rio.open(R10+'/T52WEV_20220330T025551_B02_10m.jp2')

In [45]:
with rio.open('RGB.tiff','w',driver='Gtiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()